In [1]:
import pandas as pd
test_data = pd.read_csv('/home/rohan19095/BTP/PPO/summarizationDataset/test.csv')

In [2]:
test_data.keys()

Index(['Policy_Names', 'Original_Text', 'Summary'], dtype='object')

In [3]:
# import os
# os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
# os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [4]:
!nvidia-smi


Wed Jul 26 01:15:58 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.54.03              Driver Version: 535.54.03    CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla V100-PCIE-32GB           On  | 00000000:3B:00.0 Off |                    0 |
| N/A   32C    P0              26W / 250W |      4MiB / 32768MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [5]:
text = test_data['Original_Text'][0]

In [7]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("/home/rohan19095/BTP/PPO/bartFinetunedModel")

from transformers import AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained("/home/rohan19095/BTP/SpanNer-Final/ppo/").to('cuda:0')


Some weights of the model checkpoint at /home/rohan19095/BTP/SpanNer-Final/ppo/ were not used when initializing BartForConditionalGeneration: ['v_head.summary.bias', 'v_head.summary.weight']
- This IS expected if you are initializing BartForConditionalGeneration from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BartForConditionalGeneration from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [8]:
generated_summary = []
gold_summary = test_data['Summary']
from tqdm import tqdm
for text in tqdm(test_data['Original_Text']):
    inputs = tokenizer(text,max_length=tokenizer.model_max_length, truncation=True, return_tensors="pt").input_ids.to('cuda:0')
    outputs = model.generate(inputs, max_new_tokens=1024, do_sample=False)
    generated_summary.append(tokenizer.decode(outputs[0], skip_special_tokens=True))
    

  1%|          | 3/385 [00:14<31:13,  4.91s/it]

In [ ]:
gold_summary = gold_summary.tolist()

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /tmp/ipykernel_110641/2413491558.py:1 in <module>                                                │
│                                                                                                  │
│ [Errno 2] No such file or directory: '/tmp/ipykernel_110641/2413491558.py'                       │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
AttributeError: 'list' object has no attribute 'tolist'

In [ ]:
from rouge import Rouge

def calculate_rouge_scores(gold_summaries, generated_summaries):
    rouge = Rouge()
    rouge_scores = rouge.get_scores(generated_summaries, gold_summaries, avg=True)

    return rouge_scores

# Example usage:
gold_summaries = [
    "The quick brown fox jumps over the lazy dog.",
    "The sky is blue."
]
generated_summaries = [
    "The brown fox jumps over the dog.",
    "The sky looks blue."
]

rouge_score = calculate_rouge_scores(gold_summary, generated_summary)
print(f"ROUGE: {rouge_score}")


ROUGE: {'rouge-1': {'r': 0.46678040280270316, 'p': 0.564947787889937, 'f': 0.4928236008591293}, 'rouge-2': {'r': 0.293614752966933, 'p': 0.3709711099313511, 'f': 0.3107553600727156}, 'rouge-l': {'r': 0.449756810338221, 'p': 0.5438842817537661, 'f': 0.47459738890129294}}


In [ ]:
# ROUGE: {'rouge-1': {'r': 0.5505143799376592, 'p': 0.5471579902745551, 'f': 0.531976561164638}, 'rouge-2': {'r': 0.3659014986680469, 'p': 0.36113471117447893, 'f': 0.34730192346188105}, 'rouge-l': {'r': 0.530855935516924, 'p': 0.527336835095412, 'f': 0.5128376189775129}}


In [40]:
len(gold_summary), len(generated_summary)

(385, 385)

In [37]:
gold_summary

0      We collect information from you when you respo...
1      As a consumer, you have the ability to make ch...
2      We are committed to protecting the privacy of ...
3      When you register with the Site or use any of ...
4      DSI may collect the following information when...
                             ...                        
380    During your visit, our service may gather data...
381    Datemplate, like most other websites, collects...
382    Site is not intended for or directed to person...
383    Digital Hill collects the e-mail addresses of ...
384    At www.east anime.com, the privacy of our visi...
Name: Summary, Length: 385, dtype: object

In [36]:
generated_summary

['We collect information from you when you respond to a survey or fill out a form. ##What do we collect from you may be used in one of the following ways: To improve our website. ##Do we use cookies?##We do not use cookies. ##Third party links##Occasionally, at our discretion, we may include or offer third party products or services on our Website. ##Contacting Us##If there are any questions regarding this privacy policy, you may contact us using the information below. ##Our Online Privacy Policy Only##This online privacy policy applies only to information collected through our website and not to Information collected offline.',
 "Radaris has developed this privacy policy in order to demonstrate our commitment to maintaining the privacy of the information that you provide when using Radaris' application. By submitting personal information to the Radaris web site, you agree to the terms of this Privacy Policy and expressly consent to the processing of your personal information according

In [25]:
from transformers import pipeline, set_seed, StoppingCriteriaList, StoppingCriteria
import torch


# %%

from transformers import pipeline, set_seed, StoppingCriteriaList, StoppingCriteria
import torch
class StoppingCriteriaSub(StoppingCriteria):

    def __init__(self, tokenizer=None,stops = [], encounters=1):
        super().__init__()
        self.stops = stops
        self.tokenizer=tokenizer

    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor):
        # print(input_ids)
        full_sentence = self.tokenizer.decode(input_ids[0].tolist(), clean_up_tokenization_spaces=True)
        last_token = self.tokenizer.decode(input_ids[0][-1].tolist(), clean_up_tokenization_spaces=True)
        import re
        pattern = r'\[(\w+)\]'
        sequence = re.findall(pattern, full_sentence)
        # print(sequence)
        # if(len(sequence) > 0 and sequence[-1] != 'find' and last_token == '#'):
        #     return True
        # if(last_token == '<endoftext>'):
        #     return True
        print(last_token)
        return False

In [26]:
stopping_criteria = StoppingCriteriaList([StoppingCriteriaSub(tokenizer=tokenizer,stops=['stop_words'])])


<s>
<s>
<s>
 collect
 information
 from
 you
 when
 you
 respond
 to
 a
 survey
 or
 fill
 out
 a
 form
.
.
 we
 we
 from
 you
 may
 be
 used
 in
 one
 of
 the
 following
 ways
:
:
:
:
 following
 ways
:
 our
 website
##
(
we
 continually
 strive
 to
 improve
 our
 offerings
 based
 on
 the
 information
 and
 feedback
 we
 receive
 from
 you
)
##
Do
 we
 use
 cookies
?
##
We
 do
 not
 use
 cookies
.
.
Do
Do
 disclose
 any
 information
 to
 outside
 parties
?
##
##
 questions
 regarding
 this
 privacy
 policy
,
 you
 may
 contact
 us
 using
 the
 information
 below
.
 ##
Online
 Privacy
 Policy
 Only
##
This
 online
 privacy
 policy
 applies
 only
 to
 information
 collected
 through
 our
 website
 and
 not
 to
 information
 information
.
 ##
http


In [14]:
tokenizer.decode(outputs[0], skip_special_tokens=True)

'We collect information from you when you respond to a survey or fill out a form. ##What do we collect from you may be used in one of the following ways: To improve our website. ##Do we use cookies?##We do not use cookies. ##Third party links##Occasionally, at our discretion, we may include or offer third party products or services on our Website. ##Contacting Us##If there are any questions regarding this privacy policy, you may contact us using the information below. ##Our Online Privacy Policy Only##This online privacy policy applies only to information collected through our website and not to Information collected offline.'

In [30]:
a = [1,1,1,2,2,3,4,4,5]
b = [3,3,1,1,2,6]

a_freq = {}
for element in a:
    if(element not in a_freq):
        a_freq[element] = 0
    a_freq[element] += 1
b_freq = {}
for element in b:
    if(element not in b_freq):
        b_freq[element] = 0
    b_freq[element] += 1


In [31]:
from collections import Counter
a_counter = Counter(a)
b_counter = Counter(b)

In [32]:
total_reward = 0
for entity, count in a_counter.items():
    generated_count = b_counter.get(entity, 0)
    reward = min(generated_count, count)
    total_reward += reward

In [33]:
total_reward

4

In [34]:
len(a), len(b)

(9, 6)

In [23]:
negative_reward = 0
for entity, count in b_counter.items():
    generated_count = a_counter.get(entity, 0)
    reward = min(generated_count, count)
    negative_reward += reward

In [24]:
negative_reward

4

In [25]:
a_freq, b_freq

({1: 3, 2: 2, 3: 1, 4: 2, 5: 1}, {3: 2, 1: 2, 2: 1})

In [ ]:
for element in b:
    

In [1]:
import pandas as pd
df = pd.read_csv('finetuneResults.csv')

In [8]:
print(df['generated_summary'].tolist()[3])

By using the Site or Services, you consent to this Privacy Policy. We use cookies (a small text file placed on your computer to identify your computer and browser). We also use Web beacons to improve the experience of the Site and Services, such as pre-populating your username for easier login. You can always review, correct, update, or change your Personal Information through your email settings or by contacting us at [info@day-finder.com]. You can opt out of receiving certain e-mails by unsubscribing in the emails you receive.


In [9]:
print(df['gold_summary'].tolist()[3])

When you register with the Site or use any of our Services, you may be asked to provide us with Personal Information. It is entirely optional to provide this information. If you do not provide the requested information, you may be unable to use some or all of the Site's features. To improve the Site and Services, we use cookies (a small text file placed on your computer to identify your computer and browser) and Web beacons (electronic files placed on a Web site that monitor usage). If you delete or disable cookies, some features of the Site or Services may not function properly. You have the option of making some of your Personal Information available to others. Users of the Site and commercial search engines such as Google, Yahoo!, and Bing may access the information you choose to make available. Some third-party services may provide us with information from your accounts, allowing us to improve and personalise your experience on the Site. When you visit our Site, we may allow third-

In [112]:
import json
with open('/home/rohan19095/BTP/SpanNer-Final/nerResult_3.json', 'r') as f:
    data = json.load(f)

In [113]:
gold_ner = data['gold_ner']
generated_ner = data['generated_ppo_ner']


In [114]:
acc = 0
global_total = 0
for i in range(len(generated_ner)):
    generated_summary = generated_ner[i]
    gold_summary = gold_ner[i]
    for i in range(len(generated_summary)):
        generated_summary[i][0] = generated_summary[i][0].lower()
        generated_summary[i][1] = generated_summary[i][1].lower()
    
    for i in range(len(gold_summary)):
        gold_summary[i][0] = gold_summary[i][0].lower()
        gold_summary[i][1] = gold_summary[i][1].lower()
    correct = 0
    total = 0
    if(len(generated_summary) == 0):
        continue
    for gen_ner in generated_summary:
        for gol_ner in gold_summary:
            if(gen_ner == gol_ner):
                correct += 1
                break
        total += 1
    acc += correct
    global_total += total

acc /= global_total



In [115]:
acc, global_total / len(generated_ner)

(0.6572508096437567, 14.436363636363636)

In [32]:
normal : 0.6540932794357086
loss : 0.6812139453222975


In [ ]:
10.329
10.355
14.436
